In [1]:
import setup
setup.init_django()

In [2]:
from market import services as market_services

In [3]:
def get_stock_indicators(ticker = "AAPL", days=30):
    queryset = market_services.get_daily_stock_quotes_queryset(ticker, days=days)
    if queryset.count() == 0:
        raise Exception(f"Data for {ticker} not found")
    averages = market_services.get_daily_moving_averages(ticker, days=days, queryset=queryset)
    price_target = market_services.get_price_target(ticker, days=days, queryset=queryset)
    volume_trend = market_services.get_volume_trend(ticker, days=days, queryset=queryset)
    rsi_data = market_services.calculate_rsi(ticker, days=days, period=14)
    signals = []
    if averages.get('ma_5') > averages.get('ma_20'):
        signals.append(1)
    else:
        signals.append(-1)
    if price_target.get('current_price') < price_target.get('conservative_target'):
        signals.append(1)
    else:
        signals.append(-1)
    if volume_trend.get("volume_change_percent") > 20:
        signals.append(1)
    elif volume_trend.get("volume_change_percent") < -20:
        signals.append(-1)
    else:
        signals.append(0)
    rsi = rsi_data.get('rsi')
    if rsi > 70:
        signals.append(-1)  # Overbought
    elif rsi < 30:
        signals.append(1) # Oversold
    else:
        signals.append(0)
    return {
        "score": sum(signals),
        "ticker": ticker,
        "indicators": {
            **averages,
            **price_target,
            **volume_trend,
            **rsi_data,
        }
        
    }

In [4]:
results = get_stock_indicators(ticker='AAPL')
score = results.get("score")
results

{'score': -1,
 'ticker': 'AAPL',
 'indicators': {'ma_5': 235.318,
  'ma_20': 245.604,
  'current_price': 233.78,
  'conservative_target': 243.6471,
  'aggressive_target': 249.7429,
  'average_price': 247.0319,
  'avg_volume': 2452.6923076923076,
  'latest_volume': 603,
  'volume_change_percent': -75.41477183628665,
  'rsi': 40.509,
  'avg_gain': 0.9257,
  'avg_loss': 1.3595,
  'period': 14,
  'days': 30}}

In [16]:
import json
import requests
from decouple import config
import os
os.chdir("../")

In [33]:
from decouple import config, AutoConfig
import os
os.chdir("../")
print("Current Working Directory:", os.getcwd())
print("Files in Current Directory:", os.listdir(os.getcwd()))
config = AutoConfig(search_path="/home/harry/StockTradingbot") 

DEEPSEEK_API_KEY = config("DEEPSEEK_API_KEY", default=None)

print("DEEPSEEK_API_KEY:", DEEPSEEK_API_KEY)

assert DEEPSEEK_API_KEY is not None, "DEEPSEEK_API_KEY is not set in the .env file"

Current Working Directory: /
Files in Current Directory: ['media', 'dev', 'sys', 'root', 'bin', 'lost+found', 'run', 'boot', 'usr', 'etc', 'var', 'opt', 'mnt', 'srv', 'lib', 'snap', 'home', 'Docker', 'init', 'lib32', 'proc', 'libx32', 'sbin', 'lib64', 'tmp']
DEEPSEEK_API_KEY: sk-aabd11be6d394f5d9eb2e95e8ef2b7ef


In [34]:
results_as_json = json.dumps(results)
results_as_json

'{"score": -1, "ticker": "AAPL", "indicators": {"ma_5": 235.318, "ma_20": 245.604, "current_price": 233.78, "conservative_target": 243.6471, "aggressive_target": 249.7429, "average_price": 247.0319, "avg_volume": 2452.6923076923076, "latest_volume": 603, "volume_change_percent": -75.41477183628665, "rsi": 40.509, "avg_gain": 0.9257, "avg_loss": 1.3595, "period": 14, "days": 30}}'

In [42]:
## if you have OpenAI API Key:
'''
OPENAI_API_KEY=config("OPENAI_API_KEY", default=None)
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recommendation",
            "schema": {
                "type": "object",
                "properties": {
                    "buy": {
                        "description": "Recommend to buy stock",
                        "type": "boolean"
                    },
                    "sell": {
                        "description": "Recommend to sell stock",
                        "type": "boolean"
                    },
                    "hold": {
                        "description": "Recommend to hold stock",
                        "type": "boolean"
                    },
                    "explanation": {
                        "description": "Explanation of reasoning in 1 or 2 sentences",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

result = json.loads(response.choices[0].message.content)
result

'''


'\nOPENAI_API_KEY=config("OPENAI_API_KEY", default=None)\nfrom openai import OpenAI\nclient = OpenAI(api_key=OPENAI_API_KEY)\n\n\nresponse = client.chat.completions.create(\n    model="gpt-4o-mini",\n    messages=[\n        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},\n        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}\n    ],\n    response_format={\n        "type": "json_schema",\n        "json_schema": {\n            "name": "recommendation",\n            "schema": {\n                "type": "object",\n                "properties": {\n                    "buy": {\n                        "description": "Recommend to buy stock",\n                        "type": "boolean"\n                    },\n                    "sell": {\n                        "description": "Recommend to sell stock",\n                        "type": "boolean"\n                    },\n                  

In [ ]:
## for DeepSeek Api
from openai import OpenAI

client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are an expert at analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={"type": "json_object"},  # Use "json_object" instead of "json_schema"
)

print(response.choices[0].message.content)

In [48]:
result = json.loads(response.choices[0].message.content)
result

AttributeError: 'Response' object has no attribute 'choices'

In [ ]:
result.get('hold') is True